In [2]:
import sys
sys.path.append('../kaggle_prediction_library/') 

In [10]:
import feature_engineering
import preprocess

In [4]:
class FinalRanking(feature_engineering.FeatureEng):

    def __init__(self, rankings_df, system):
        self.rankings_df = rankings_df
        self.system = system

    def process(self):

        last_day = self.rankings_df.groupby('Season').agg({'RankingDayNum':'max'}).rename(
                                    columns = {'RankingDayNum': 'last_day'})


        end_rankings = self.rankings_df.join(last_day, on = 'Season')
        end_rankings = end_rankings[ (end_rankings.RankingDayNum == end_rankings.last_day)
                                    & (end_rankings.SystemName ==self.system)]
        end_rankings = end_rankings.groupby(['TeamID', 'Season']).agg({'OrdinalRank':np.mean})
        end_rankings.reset_index(inplace = True)
        end_rankings.columns = ['TeamID', 'Season', 'avg_rank']
        end_rankings['final_rank'] = 100-4*np.log(end_rankings['avg_rank']+1)-end_rankings['avg_rank']/22
        
        return end_rankings[['TeamID', 'Season', 'final_rank']]

In [8]:
# For now we can use this, but we may need to recreate this once the teams have been announced
# The new version should only include feasible tourney matchups to make life easier
import pandas as pd
sub_df = pd.read_csv('../production_notebooks/SampleSubmission2024.csv', index_col=False)

In [39]:
regular_season_results = pd.read_csv('../data/MRegularSeasonDetailedResults.csv')
detailed_tourney_results = pd.read_csv('../data/MNCAATourneyDetailedResults.csv')
rankings = pd.read_csv('../data/MMasseyOrdinals.csv')
seeds = pd.read_csv('../data/MNCAATourneySeeds.csv')

# kp_rankings = pd.read_csv('../data/kenpom_pre_tourney_snapshot.csv')

regular_season_results_w = pd.read_csv('../data/WRegularSeasonDetailedResults.csv')
detailed_tourney_results_w = pd.read_csv('../data/WNCAATourneyDetailedResults.csv')

mteams = pd.read_csv('../data/MTeams.csv')
wteams = pd.read_csv('../data/WTeams.csv')

seeds_w = pd.read_csv('../data/WNCAATourneySeeds.csv')

# M538 = pd.read_csv('../data/M538.csv')
# W538 = pd.read_csv('../data/W538.csv')

seed_round = pd.read_csv("../data/MNCAATourneySeedRoundSlots.csv")
seeds = pd.read_csv("../data/MNCAATourneySeeds.csv")

first_round_odds_data = pd.read_csv('../data/first_round_odds_2024.csv')



In [40]:
to_predict_mens, to_predict_womens, regular_season_games, regular_season_games_w = preprocess.full_setup(detailed_tourney_results, regular_season_results,
               detailed_tourney_results_w, regular_season_results_w,
               sub_df, mteams)

In [41]:
to_predict_mens = to_predict_mens[to_predict_mens.Season <= 2023]

In [42]:
to_predict_mens = feature_engineering.FinalRanking(rankings_df=rankings, system='SAG').add(to_predict_mens)


In [43]:
import tqdm

In [44]:
corrs = []

for i in tqdm.tqdm(rankings[rankings.Season == 2024].SystemName.unique()):
    
    try:
        to_predict_mens_tmp = feature_engineering.FinalRanking(rankings_df=rankings, system=i).add(to_predict_mens)
        corr = to_predict_mens_tmp[["t1_final_rank_x", "t1_final_rank_y"]].corr().iloc[0,1]
        corrs.append((i ,corr))

    except Exception:
        pass



  0%|          | 0/60 [00:00<?, ?it/s]

100%|██████████| 60/60 [01:24<00:00,  1.41s/it]


In [45]:
tmp_df = pd.DataFrame(corrs, columns=["system", "corr"])

In [53]:
rankings_dates = rankings[rankings.SystemName.isin(rankings[rankings.Season == 2024].SystemName.unique())].groupby("SystemName").Season.agg(["min", "max"]).reset_index()

In [55]:
rankings_dates.columns = ["system", "min", "max"]

In [56]:
tmp_df.merge(rankings_dates, on="system").sort_values("corr", ascending=False)

,system,corr,min,max
21,TRP,0.992046,2014,2024
10,INC,0.988305,2019,2024
42,WLK,0.987103,2003,2024
49,PIR,0.984640,2019,2024
46,SRS,0.983244,2021,2024
5,DOK,0.981053,2006,2024
20,TRK,0.981043,2015,2024
1,BNZ,0.980968,2020,2024
2,BWE,0.980565,2015,2024
7,EBP,0.980192,2012,2024


In [ ]:
POM 

In [ ]:
# may want to find a linear combination that produces SAG even closer?

In [ ]:
to_predict_mens = feature_engineering.FinalRanking(rankings_df=rankings, system='SAG').add(to_predict_mens)
